# Meta-analysis on log-variance ratio

### Simulation parameters

In [53]:
suppressPackageStartupMessages(library(meta))

Niter <- 10000          # Number of iterations for bootstrap simulation
Nstud <- 10             # Number of studies in the meta-analysis
ss1 <- 20:40            # Different possible number of cases
ss2 <- 5:30             # Different possible number of controls
mu1 <- 10               # True mean for cases
mu2 <- 10               # True mean for controls
sigma1 <- 1.2           # True standard deviation for cases
sigma2 <- 1             # True standard deviation for controls

### Samples generation

In [54]:
n1 <- sample(ss1, Nstud, rep=T)     # Number of cases for each study
n2 <- sample(ss2, Nstud, rep=T)     # Number of controls for each study

ind1 <- lapply(n1, rnorm, mu1, sigma1)      # Cases generation
ind2 <- lapply(n2, rnorm, mu2, sigma2)      # Controls generation
sd1 <- sapply(ind1, sd)                     # Estimated standard deviations for cases of each study
sd2 <- sapply(ind2, sd)                     # Estimated standard deviations for controls of each study

### F-test for comparison two variances

In [55]:
Fr <- sd1^2 / sd2^2                                                 # Fisher ratios
df1 <- n1 - 1                                                       # Degrees of freedom for F-test
df2 <- n2 - 1
f.pvalues <- 1 - abs(pf(Fr, df1, df2) - 0.5) * 2                    # F-test p-values
alpha <- .05
ci <- cbind(Fr/qf(1-alpha/2, df1, df2), Fr/qf(alpha/2, df1, df2))   # 95% Confidence intervals for variance ratios
data.frame(study=1:Nstud, cases=n1, controls=n2, estimate=Fr, lower.95=ci[,1], upper.95=ci[,2], pval=f.pvalues)

study,cases,controls,estimate,lower.95,upper.95,pval
1,23,24,1.8848013,0.8105226,4.418297,0.13867008
2,23,12,1.3381884,0.4185702,3.541987,0.63052687
3,24,24,1.9907306,0.8611765,4.601854,0.10575130
4,28,15,0.8789126,0.3187168,2.104866,0.74520592
5,34,22,0.9191367,0.4020617,1.959113,0.80907985
6,28,9,0.9869077,0.2519044,2.671950,0.89910365
7,23,29,1.5539479,0.7061529,3.557181,0.26937288
8,23,21,2.0175320,0.8290070,4.819849,0.11963258
9,20,17,2.4768702,0.9180296,6.416713,0.07229135
10,26,12,2.8771494,0.9100170,7.366377,0.07043425


### Meta-analysis on ratios of variances

In [56]:
TE <- log(Fr) - digamma(df1/2) + digamma(df2/2) + log(df1/df2)
TEse <- sqrt(trigamma(df1/2) + trigamma(df2/2))
meta <- metagen(TE, TEse)
cat(sprintf("Variance ratio estimate (real value: %g):", sigma1^2/sigma2^2))
with(meta, data.frame(method=c("fixed", "random"), estimate=exp(c(TE.fixed, TE.random)/2),
        lower.95=exp(c(lower.fixed, lower.random)/2),
        upper.95=exp(c(upper.fixed, upper.random)/2), pval=c(pval.fixed, pval.random)))

Variance ratio estimate (real value: 1.44):

method,estimate,lower.95,upper.95,pval
fixed,1.239225,1.07264,1.431681,0.003591341
random,1.239225,1.07264,1.431681,0.003591341


### Monte Carlo simulation

In [61]:
metas <- replicate(Niter, {
	n1 <- sample(ss1, Nstud, rep=T)
	n2 <- sample(ss2, Nstud, rep=T)
	
	ind1 <- lapply(n1, rnorm, mu1, sigma1)
	ind2 <- lapply(n2, rnorm, mu2, sigma2)
	sd1 <- sapply(ind1, sd)
	sd2 <- sapply(ind2, sd)
	
	Fr <- sd1^2 / sd2^2
	df1 <- n1 - 1
	df2 <- n2 - 1
    
	TE1 <- log(Fr) - digamma(df1/2) + digamma(df2/2) + log(df1/df2)     # Correcting for bias
	TE2 <- log(Fr)                                                      # Not correcting for bias
	TEse <- sqrt(trigamma(df1/2) + trigamma(df2/2))
	list(nobias=metagen(TE1, TEse, n.e=n1, n.c=n2), bias=metagen(TE2, TEse, n.e=n1, n.c=n2))
}, simplify=F)

### Distribution of estimations
<b>mean:</b> mean log-variance ratio estimated by meta-analysis<br>
<b>sd:</b> standard deviation of the estimations, which corresponds to the standard error of one estimation<br>
<b>sem:</b> standard error for the mean of the estimations<br>
<b>p.value:</b> p-value for the existence of a bias in the estimations<br>
<b>fixed:</b> fixed effect model<br>
<b>random:</b> random effects model

In [73]:
metas1 <- lapply(metas, "[[", "nobias")
metas2 <- lapply(metas, "[[", "bias")
Ne <- colSums(sapply(metas1, '[[', 'n.e'))
Nc <- colSums(sapply(metas1, '[[', 'n.c'))
lvr <- 2*log(sigma1/sigma2)
cat(sprintf("Total number of cases by meta-analysis: %g ± %g\n", mean(Ne), sd(Ne)))
cat(sprintf("Total number of cases by meta-analysis: %g ± %g\n\n", mean(Nc), sd(Nc)))
cat(sprintf("Real log-variance ratio: %g\n\n", lvr))
cat("Combined estimations with bias correction:\n")
TEs1.fixed <- sapply(metas1, "[[", "TE.fixed")
TEs1.random <- sapply(metas1, "[[", "TE.random")
TEs1.mean <- c(mean(TEs1.fixed), mean(TEs1.random))
TEs1.sd <- c(sd(TEs1.fixed), sd(TEs1.random))
TEs1.sem <- TEs1.sd / sqrt(length(TEs1.fixed))
TEs1.p <- 1-2*abs(pnorm(TEs1.mean, lvr, TEs1.sem)-.5)
data.frame(mean = TEs1.mean, sd=TEs1.sd, sem=TEs1.sem, p.value=TEs1.p, row.names=c("fixed", "random"))
cat("Combined estimations without bias correction:\n")
TEs2.fixed <- sapply(metas2, "[[", "TE.fixed")
TEs2.random <- sapply(metas2, "[[", "TE.random")
TEs2.mean <- c(mean(TEs2.fixed), mean(TEs2.random))
TEs2.sd <- c(sd(TEs2.fixed), sd(TEs2.random))
TEs2.sem <- TEs2.sd / sqrt(length(TEs2.fixed))
TEs2.p <- 1-2*abs(pnorm(TEs2.mean, lvr, TEs2.sem)-.5)
data.frame(mean = TEs2.mean, sd=TEs2.sd, sem=TEs2.sem, p.value=TEs2.p, row.names=c("fixed", "random"))


Total number of cases by meta-analysis: 300.098 ± 19.5001
Total number of cases by meta-analysis: 174.579 ± 23.6432

Real log-variance ratio: 0.364643

Combined estimations with bias correction:


,mean,sd,sem,p.value
fixed,0.3622764,0.1470438,0.001470438,0.1075013
random,0.3634180,0.1480556,0.001480556,0.4079570


Combined estimations without bias correction:


,mean,sd,sem,p.value
fixed,0.3939271,0.1473636,0.001473636,0
random,0.3974276,0.1490018,0.001490018,0


### Statistical power of estimations (rate of log-variance ratios estimated significantly different from 0)

In [71]:
power1.fixed <- mean(sapply(metas1, "[[", "pval.fixed") <= alpha)
power1.random <- mean(sapply(metas1, "[[", "pval.random") <= alpha)
power2.fixed <- mean(sapply(metas2, "[[", "pval.fixed") <= alpha)
power2.random <- mean(sapply(metas2, "[[", "pval.random") <= alpha)
data.frame(nobias=c(power1.fixed, power1.random), bias=c(power2.fixed, power2.random), row.names=c("fixed", "random"))

,nobias,bias
fixed,0.6940,0.7619
random,0.6285,0.7084


### Frequency of incorrect estimations (Rate of estimated 95% confidence intervals not including real value)

In [72]:
lvr <- 2*log(sigma1/sigma2)
fie1.fixed <- mean(sapply(metas1, "[[", "lower.fixed") > lvr | sapply(metas1, "[[", "upper.fixed") < lvr)
fie1.random <- mean(sapply(metas1, "[[", "lower.random") > lvr | sapply(metas1, "[[", "upper.random") < lvr)
fie2.fixed <- mean(sapply(metas2, "[[", "lower.fixed") > lvr | sapply(metas2, "[[", "upper.fixed") < lvr)
fie2.random <- mean(sapply(metas2, "[[", "lower.random") > lvr | sapply(metas2, "[[", "upper.random") < lvr)
data.frame(nobias=c(fie1.fixed, fie1.random), bias=c(fie2.fixed, fie2.random), row.names=c("fixed", "random"))

,nobias,bias
fixed,0.0507,0.0536
random,0.0384,0.0406
